In [1]:
import pandas as pd
import glob
import os

# ukeireディレクトリのパス
ukeire_dir = "/app/app/data/ukeire/"

# ukeire内の全csvファイルを取得
csv_files = glob.glob(os.path.join(ukeire_dir, "*.csv"))

# 全csvを読み込んでリスト化
df_list = [pd.read_csv(f) for f in csv_files]

# 全てのDataFrameを縦方向に結合（カラムが異なる場合はNaNで補完）
df = pd.concat(df_list, ignore_index=True)

# 結果のDataFrameを表示
df.head()

,伝票日付,売上日付,支払日付,業者CD,業者名,伝票区分CD,伝票区分名,品名CD,品名,正味重量,...,荷降現場名,運搬業者CD,運搬業者名,取引先CD,取引先名,マニ種類CD,マニ種類名,マニフェスト番号,営業担当者CD,営業担当者名
0,2025/06/01(日),2025/06/01(日),2025/06/01(日),144,H&N,1,売上,1,混合廃棄物A,"1,110",...,新木場ﾘｻｲｸﾙ工場,144.0,H&N,000115,H&N,1.0,紙,NaN,2,渡辺
1,2025/06/01(日),2025/06/01(日),2025/06/01(日),740,ﾌﾛﾝﾃｨｱ,1,売上,1,混合廃棄物A,910,...,新木場ﾘｻｲｸﾙ工場,740.0,ﾌﾛﾝﾃｨｱ,000570,ﾌﾛﾝﾃｨｱ（渡辺・足立区）,1.0,紙,NaN,2,渡辺
2,2025/06/01(日),2025/06/01(日),2025/06/01(日),740,ﾌﾛﾝﾃｨｱ,1,売上,1,混合廃棄物A,"1,230",...,新木場ﾘｻｲｸﾙ工場,740.0,ﾌﾛﾝﾃｨｱ,000570,ﾌﾛﾝﾃｨｱ（渡辺・足立区）,1.0,紙,NaN,2,渡辺
3,2025/06/01(日),2025/06/01(日),2025/06/01(日),740,ﾌﾛﾝﾃｨｱ,1,売上,1,混合廃棄物A,"1,950",...,新木場ﾘｻｲｸﾙ工場,740.0,ﾌﾛﾝﾃｨｱ,000570,ﾌﾛﾝﾃｨｱ（渡辺・足立区）,1.0,紙,NaN,2,渡辺
4,2025/06/01(日),2025/06/01(日),2025/06/01(日),751,まごのて,1,売上,1,混合廃棄物A,870,...,新木場ﾘｻｲｸﾙ工場,751.0,まごのて,000580,まごのて,2.0,電子,NaN,6,川部


In [ ]:

import yaml
# 2. YAMLからマッピング辞書を取得
with open("/app/config/header_mapping.yaml", encoding="utf-8") as f:
    mapping_yaml = yaml.safe_load(f)

mapping = mapping_yaml["品名一覧"]["columns"]

# 4. DataFrameのカラム名を英語にリネーム
df = df.rename(columns=mapping)
df = df.drop(['deleted', 'qualified_return', 'mixed', 'mixed_1', 'mixed_2',], axis=1)
# 'cd' を含むカラムを抽出（大文字・小文字どちらも対象にする場合は .str.lower()）
cd_columns = [col for col in df.columns if 'cd' in col.lower()]

# 各該当カラムを int に変換（NaN があると int に変換できないので注意）
for col in cd_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
df["tax_type"]=df["tax_type"].astype("Int64")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from app.db.database import get_engine
# 接続エンジン作成
engine = get_engine()
# df = pd.read_sql("SELECT * FROM config.clients", engine)

df.to_sql(
    name="items",       # テーブル名（業者一覧ならvendorsなど）
    con=engine,
    schema="config",      # スキーマ
    if_exists="replace",  # テーブル全置き換えなら'replace'（追記なら'append'）
    index=False
)